This notebook demos two tranformers, which broadly aim at producing abstract representations of an utterance in terms of its phrasing and its rhetorical intent: 

* The `PhrasingMotifs` transformer extracts representations of utterances in terms of how they are phrased;
* The `PromptTypes` transformer computes latent representations of utterances in terms of their rhetorical intention -- the _responses_ they aim at prompting -- and assigns utterances to different (automatically-inferred) types of intentions.

It also demos some additional transformers used in preprocessing steps.



Together, these transformers implement the methodology detailed in the [paper](http://www.cs.cornell.edu/~cristian/Asking_too_much.html), 

```
Asking Too Much? The Rhetorical Role of Questions in Political Discourse 
Justine Zhang, Arthur Spirling, Cristian Danescu-Niculescu-Mizil
Proceedings of EMNLP 2017
```

ConvoKit also includes an end-to-end implementation, `PromptTypesWrapper`, that runs the transformers one after another, and handles the particular pre-processing steps found in the paper. See TODO LINK for a demonstration of this end-to-end transformer.

This is a really clear example of a method which reflects both good (we think) ideas and somewhat ad-hoc implementation decisions. As such, there are lots of options and potential variations to consider (beyond the deeper question of what phrasings and intentions even are) -- I'll detail these as I go along.

Note that due to small methodological tweaks and changes in the random seed, the particular output of the transformers as presently implemented may not totally match the output from the paper, but the broad types of questions returned are comparable.

## Preliminaries

First we load the corpus. We will examine a dataset of questions from question periods that take place in the British House of Commons (also detailed in the paper). 

In [1]:
import convokit

In [2]:
ROOT_DIR = '/kitchen/clean-corpora/parliament-corpus/'

We'll load the corpus, plus some pre-computed dependency parses (see TODO LINK for a demonstration of how to get these parses on your own; for this dataset they should be TODO included with our release).

In [3]:
corpus = convokit.Corpus(ROOT_DIR)
corpus.load_info('utterance',['parsed'])

In [4]:
VERBOSITY = 10000

Our specific goal, which we'll use ConvoKit to accomplish, is to produce an abstract representation of questions asked by members of parliament, in terms of:

* how they are phrased: what phrasing, or lexico-syntatic "motif", does a question have? 
* their rhetorical intention: what's the intent of the asker -- which we take to mean the response the asker aims to prompt? 

In other words, what are the different types of questions people ask in parliament?

Here's an example of an utterance:

In [5]:
test_utt_id = '1997-01-27a.4.0'
utt = corpus.get_utterance(test_utt_id)

In [6]:
utt.text

"Does my right hon Friend agree that last week 's statement about a replacement royal yacht has been widely welcomed ? Does he agree also that , ideally , Britannia should become the centrepiece of the millennium project in Portsmouth harbour , spanning Gosport and Portsmouth ? I am sure that that idea would prove very popular . As to plans for a new yacht , does my right hon Friend share my distaste for the Opposition 's tactics ? They had every opportunity to express their grudging and negative attitude during the past two years when the project was under discussion ."

To state our goals more precisely:

* For each _sentence_ that has a question (all but the last), we want to come up with a representation of the sentence's phrasing. Intuitively, for instance, the first two sentences sound like they could both be thought of as a "Does X agree that Y?" -- whether Y is asking about a yacht or a harbour. 
* For each utterance, we want to come up with a representation of the utterance's rhetorical intent. Intuitively, all the questions could be construed as asking if the answerer is in agreement with the asker -- whether they "agree" with the opinion or "share" the opinion. We might think of this as being an example of an "agreeing" type of question.

Intuitively, if we want to get at this higher level of abstraction, we have to look beyond the particular n-grams: it doesn't seem plausible that there is a meaningful type of question about yachts (unless our specific context is the parliamentary subcommittee on yachts). 



## Preprocessing step: Arcs

One place to start is to look at the structural "skeleton" of sentences -- i.e., its dependency parse. Thus, we are first going to provide a representation of questions in terms of their dependency parse by extracting all the parent-to-child token edges, or "arcs". We will use the `TextToArcs` class to do this:

In [8]:
from convokit.text_processing import TextToArcs

`get_arcs` is a transformer (actually a `TextProcessor`) that will read the dependency parse of an utterance and write the resultant arcs to a field called `'arcs'`:

In [9]:
get_arcs = TextToArcs('arcs', verbosity=VERBOSITY)
corpus = get_arcs.transform(corpus)

10000/433787 utterances processed
20000/433787 utterances processed
30000/433787 utterances processed
40000/433787 utterances processed
50000/433787 utterances processed
60000/433787 utterances processed
70000/433787 utterances processed
80000/433787 utterances processed
90000/433787 utterances processed
100000/433787 utterances processed
110000/433787 utterances processed
120000/433787 utterances processed
130000/433787 utterances processed
140000/433787 utterances processed
150000/433787 utterances processed
160000/433787 utterances processed
170000/433787 utterances processed
180000/433787 utterances processed
190000/433787 utterances processed
200000/433787 utterances processed
210000/433787 utterances processed
220000/433787 utterances processed
230000/433787 utterances processed
240000/433787 utterances processed
250000/433787 utterances processed
260000/433787 utterances processed
270000/433787 utterances processed
280000/433787 utterances processed
290000/433787 utterances proc

`'arcs'` is a list where each element corresponds to a sentence in the utterance. Each sentence is represented in terms of its arcs, in a space-separated string. 

Each arc, in turn, can be read as follows:

* `x_y` means that `x` is the parent and `y` is the child token (e.g., `agree_does` = `agree --> does`)
* `x_*` means that `x` is a token with at least one descendant, which we do not resolve (this is roughly like bigrams backing off to unigrams)
* `x>y` means that `x` and `y` are the first two tokens in the sentence (the decision here was that how the sentence starts is a signal of "phrasing structure" on par with the dependency tree structure)
* `x>*` means that `x` is the first token in the sentence. 

In [10]:
utt.get_info('arcs')

["'s_* a_* about_* about_yacht agree_* agree_does agree_hon agree_welcomed been_* does>* does>my does_* friend_* has_* hon_* hon_friend hon_my hon_right last_* my_* replacement_* right_* royal_* statement_* statement_about statement_week that_* week_'s week_* week_last welcomed_* welcomed_been welcomed_has welcomed_statement welcomed_that welcomed_widely widely_* yacht_* yacht_a yacht_replacement yacht_royal",
 'agree_* agree_also agree_become agree_does agree_he also_* become_* become_britannia become_centrepiece become_ideally become_should become_spanning become_that britannia_* centrepiece_* centrepiece_of centrepiece_the does>* does>he does_* gosport_* harbour_* harbour_portsmouth he_* ideally_* in_* in_harbour millennium_* of_* of_project portsmouth_* project_* project_in project_millennium project_the should_* spanning_* spanning_gosport that_* the_*',
 'am_* am_i am_sure i>* i_* idea_* idea_that popular_* popular_very prove_* prove_idea prove_popular prove_that prove_would sure

### Further preprocessing: cleaned-up arcs

At this point, while we've got the methodology to start making sense of the dependency tree, we arguably haven't progressed beyond producing fancy bigram representations of sentences. One problem is perhaps that the default arc extraction is a bit too permissive -- it gives us _all_ of the arcs. We might not want this for a few reasons:

* We only want to learn about question phrasings; we don't actually care about non-question sentences.
* The structure of a question might be best encapsulated by the arcs that go out of the _root_ of the tree; as you get further down we might end up with less structural and more content-specific representations.
* Likewise, the particular _nouns_ used (e.g., `yacht`) might not be good descriptions of the more abstract phrasing pattern.

All of these points are debatable, and the resultant modules I'll show below hopefully allow you to play around with them. Taking these point as is for now, though, we'll do the following.

In [11]:
from convokit.phrasing_motifs import CensorNouns, QuestionSentences
from convokit.convokitPipeline import ConvokitPipeline

We will actually create a pipeline to extract the arcs we want. This pipeline has the following components, in order:

* `CensorNouns`: a transformer that removes all the nouns and pronouns from a dependency parse. This transformer also collapses constructions like `What time [is it]` into `What [is it]`.
* `TextToArcs`: calling the arc extractor from above with an extra parameter: `root_only=True` which will only extract arcs attached to the root (in addition to the first two tokens, though this is also tunable by passing in parameter `use_start=True`).
* `QuestionSentences`: a transformer that, given utterance fields consisting of a list of sentences, removes all the sentences which contain question marks. Here, we pass an extra parameter `input_filter=question_filter`, telling it to ignore utterances which aren't listed in the Corpus as questions (i.e., if a player asks a question, we'll discount this, since it's not labeled in the Corpus as a reporter question). 
    * (you may wonder how this transformer can tell whether a sentence has a question mark in it, given that the output of `TextToArcs` doesn't have any punctuation. Under the hood, `QuestionSentences` looks at the dependency parse of the sentence and checks whether the last token is a question.)
    * `QuestionSentences` also omits any sentences which don't begin in capital letters. To turn this off, pass parameter `use_caps=False`.

In [12]:
def question_filter(utt, aux_input={}):
    return utt.meta['is_question']

In [13]:
q_arc_pipe = ConvokitPipeline([
    ('censor_nouns', CensorNouns('parsed_censored', verbosity=VERBOSITY)),
    ('shallow_arcs', TextToArcs('arcs_censored', input_field='parsed_censored', 
                               root_only=True, verbosity=VERBOSITY)),
    ('question_sentence_filter', QuestionSentences('question_arcs', input_field='arcs_censored',
                                         input_filter=question_filter, verbosity=VERBOSITY))
])

In [14]:
corpus = q_arc_pipe.transform(corpus)

10000/433787 utterances processed
20000/433787 utterances processed
30000/433787 utterances processed
40000/433787 utterances processed
50000/433787 utterances processed
60000/433787 utterances processed
70000/433787 utterances processed
80000/433787 utterances processed
90000/433787 utterances processed
100000/433787 utterances processed
110000/433787 utterances processed
120000/433787 utterances processed
130000/433787 utterances processed
140000/433787 utterances processed
150000/433787 utterances processed
160000/433787 utterances processed
170000/433787 utterances processed
180000/433787 utterances processed
190000/433787 utterances processed
200000/433787 utterances processed
210000/433787 utterances processed
220000/433787 utterances processed
230000/433787 utterances processed
240000/433787 utterances processed
250000/433787 utterances processed
260000/433787 utterances processed
270000/433787 utterances processed
280000/433787 utterances processed
290000/433787 utterances proc

This pipeline results in a more minimalistic representation of utterances, in terms of just the arcs at the root of dependency trees, just the questions, and no nouns:

In [15]:
utt.get_info('question_arcs')

['agree_* agree_does agree_welcomed does>*',
 'agree_* agree_also agree_become agree_does does>*',
 'as>* as>to share_* share_does']

Here's another example:

In [97]:
test_utt_id_1 = '2015-06-09c.1041.5'
utt1 = corpus.get_utterance(test_utt_id_1)

In [98]:
utt1.text

'Given what the Foreign Secretary has said about the importance of the Iran discussions on the nuclear agreement , what is he doing to ensure greater clarity about the baselines , the extent of the inspection regime and the consequences of infringement ? Given that the agreement will allow advanced centrifuge , the infringements might arrive a little earlier than anticipated .'

In [99]:
utt1.get_info('question_arcs')

['doing_* doing_ensure doing_given doing_is doing_what given>* given>what']

## Phrasing motifs

Finally, to arrive at our representation of phrasings, we can go one further level of abstraction. In short, some of these arcs feel less fully-specified than others. While `agree_does` sounds like it hints at a coherent question, `doing_is` seems like it's not meaningful until you consider that it occurs in the same sentence as `doing_ensure` (i.e., "_what is the Government doing to ensure...?_")

Our intuition is to think of phrasings as frequently-cooccurring sets of multiple arcs. To extract these frequent arc-sets (which may remind you of the data mining idea of extracting frequent itemsets) we will use the `PhrasingMotifs` class.

In [16]:
from convokit.phrasing_motifs import PhrasingMotifs

In [17]:
pm_model = PhrasingMotifs('motifs','question_arcs',min_support=100,fit_filter=question_filter,
                          verbosity=VERBOSITY)

Here, `pm_model` will:

* extract all sets of arcs, as read from the `question_arcs` field, which occur at least 50 times in a corpus. These frequently-occurring arc sets will constitute the set, or "vocabulary", of phrasings.
* write the resultant output -- the phrasings that an utterance contains -- to a field called `question_motifs`. 

On the latter point, `pm_model` will only transform (i.e., label phrasings for) utterances which are questions, i.e., `question_filter(utterance) = True`. That is, in both the train and transform steps, we totally ignore non-questions.

Note that the phrasings learned by `pm_model` are therefore _corpus-specific_ -- different corpora may have different frequently-occurring sets, resulting in different vocabularies of phrasings. For instance, you wouldn't expect people in the British House of Commons to ask questions that sound like questions asked to tennis players. In this respect, think of `PhrasingMotifs` like models from scikit learn (e.g., `LogisticRegression`) -- it is fit to a particular dataset:

In [18]:
pm_model.fit(corpus)

counting frequent itemsets for 325339 sets
	first pass: counting itemsets up to and including 5 items large
	first pass: 10000/325339 sets processed
	first pass: 20000/325339 sets processed
	first pass: 30000/325339 sets processed
	first pass: 40000/325339 sets processed
	first pass: 50000/325339 sets processed
	first pass: 60000/325339 sets processed
	first pass: 70000/325339 sets processed
	first pass: 80000/325339 sets processed
	first pass: 90000/325339 sets processed
	first pass: 100000/325339 sets processed
	first pass: 110000/325339 sets processed
	first pass: 120000/325339 sets processed
	first pass: 130000/325339 sets processed
	first pass: 140000/325339 sets processed
	first pass: 150000/325339 sets processed
	first pass: 160000/325339 sets processed
	first pass: 170000/325339 sets processed
	first pass: 180000/325339 sets processed
	first pass: 190000/325339 sets processed
	first pass: 200000/325339 sets processed
	first pass: 210000/325339 sets processed
	first pass: 220000

Here are the most common phrasings and how often they occur in the data (in # of sentences). Note that `('*',)` denotes the null phrasing -- i.e., it encapsulates sentences with _any_ root word. 

In [19]:
pm_model.print_top_phrasings(25)

('*',) 325339
('will>*',) 67920
('does>*',) 59959
('is_*',) 57904
('is>*',) 45238
('is>*', 'is_*') 42850
('agree_*',) 36085
('agree_does',) 33685
('agree_*', 'agree_does') 33685
('agree_*', 'does>*') 30009
('agree_does', 'does>*') 29984
('agree_*', 'agree_does', 'does>*') 29984
('is_aware',) 22049
('is_*', 'is_aware') 22049
('is>*', 'is_aware') 20704
('is>*', 'is_*', 'is_aware') 20704
('what>*',) 20518
('is_not',) 15977
('is_*', 'is_not') 15977
('is>*', 'is_not') 13408
('is>*', 'is_*', 'is_not') 13408
('accept_*',) 11867
('agree_is',) 11059
('agree_*', 'agree_is') 11059
('agree_does', 'agree_is') 10613


Having "trained", or fitted our model, we can then use it to annotate each (question) utterance in the corpus with the phrasings this utterance contains.

In [20]:
corpus = pm_model.transform(corpus)

10000/433787 utterances processed
20000/433787 utterances processed
30000/433787 utterances processed
40000/433787 utterances processed
50000/433787 utterances processed
60000/433787 utterances processed
70000/433787 utterances processed
80000/433787 utterances processed
90000/433787 utterances processed
100000/433787 utterances processed
110000/433787 utterances processed
120000/433787 utterances processed
130000/433787 utterances processed
140000/433787 utterances processed
150000/433787 utterances processed
160000/433787 utterances processed
170000/433787 utterances processed
180000/433787 utterances processed
190000/433787 utterances processed
200000/433787 utterances processed
210000/433787 utterances processed
220000/433787 utterances processed
230000/433787 utterances processed
240000/433787 utterances processed
250000/433787 utterances processed
260000/433787 utterances processed
270000/433787 utterances processed
280000/433787 utterances processed
290000/433787 utterances proc

One thing to note here is that each sentence can and probably will have multiple phrasings it embodies. For instance, two sentences with phrasing `agree_do` and `agree_will` will also have phrasing `agree_*`. Intuitively, more finely-specified phrasings (i.e., `agree_does`) more closely specify the phrasing embodied by a sentence (we could imagine "Do you agree..." and "Will you agree..." being very different, but perhaps also more similar to each other than "Can you explain.."). 

We want to keep track of both the complete set of phrasings and the most finely-specified phrasing you can have for each utterance. Therefore, `PhrasingMotifs` actually annotates utterances with _two_ fields.

`motifs` lists all the phrasings (arcs in a phrasing motif are separated by two underscores, `'__'`):

In [21]:
utt.get_info('motifs')

['agree_* agree_*__does>* does>*',
 'agree_* agree_*__agree_also agree_*__does>* does>*',
 'as>* share_* share_*__share_does']

and `motifs__sink` lists the most finely specified _sink phrasings_ (they are "sinks" in the sense that if you think of phrasings as a directed graph where A-->B when B is a more finely-specified version of A, these sinks have no child phrasings which are contained in the utterance)

In [22]:
utt.get_info('motifs__sink')

['agree_*__does>*', 'agree_*__agree_also', 'as>* share_*__share_does']

We'll save a subset of our output to disk -- the filtered arcs, and the motifs, potentially for use in a later transformer.

In [23]:
corpus.dump_info('utterance', ['motifs', 'motifs__sink', 'arcs_censored'])

### model persistence

We can save `pm_model` to disk and later reload it, thus caching the trained model (i.e., the motifs in a corpus and the internal representation of these motifs). Here, we save the model to a `pm_model` subfolder in the corpus directory via `dump_model()`:

In [24]:
import os

In [25]:
pm_model.dump_model(os.path.join(ROOT_DIR, 'pm_model'))

writing itemset counts
writing downlinks
writing itemset to ids
writing meta information


This subfolder then stores the motifs, as well as relations between the motifs that facilitate transforming new utterances.

In [26]:
pm_model_dir = os.path.join(ROOT_DIR, 'pm_model')
!ls $pm_model_dir

downlinks.json	itemset_counts.json  itemset_to_ids.json  meta.json


Suppose we later initialize a new `PhrasingMotifs` model, `new_pm_model`.

In [27]:
new_pm_model = PhrasingMotifs('motifs_new','question_arcs',min_support=100,fit_filter=question_filter,
                          verbosity=VERBOSITY)

Calling `load_model()` then reloads the stored model from our earlier run into this new model:

In [28]:
new_pm_model.load_model(os.path.join(ROOT_DIR, 'pm_model'))

reading itemset counts
reading downlinks
reading itemset to ids
reading meta information


Just to check that we've loaded the same thing that we previously saved, we'll get the motifs in our test utterance using `new_pm_model`:

In [29]:
utt = new_pm_model.transform_utterance(utt)

This is the output from the original run:

In [30]:
utt.get_info('motifs__sink')

['agree_*__does>*', 'agree_*__agree_also', 'as>* share_*__share_does']

And we see the new output matches.

In [31]:
utt.get_info('motifs_new__sink')

['agree_*__does>*', 'agree_*__agree_also', 'as>* share_*__share_does']

### example variation: not removing the nouns

**note** this takes a while to run, and is somewhat of an extension -- you can safely skip these cells.

There are other ways to use `PhrasingMotifs` that might be more or less suited to your own application. For instance, you may wonder what happens if we do not remove the nouns (as we did with `CensorNouns` above). To try this out, we can create an alternate pipeline that uses `TextToArcs` to generate root arcs (setting argument `root_only=True`) on the original parses, not the noun-censored ones.

In [32]:
q_arc_pipe_full = ConvokitPipeline([
    ('shallow_arcs_full', TextToArcs('root_arcs', input_field='parsed', 
                               root_only=True, verbosity=VERBOSITY)),
    ('question_sentence_filter', QuestionSentences('question_arcs_full', input_field='root_arcs',
                                         input_filter=question_filter, verbosity=VERBOSITY)),

])
corpus = q_arc_pipe_full.transform(corpus)


10000/433787 utterances processed
20000/433787 utterances processed
30000/433787 utterances processed
40000/433787 utterances processed
50000/433787 utterances processed
60000/433787 utterances processed
70000/433787 utterances processed
80000/433787 utterances processed
90000/433787 utterances processed
100000/433787 utterances processed
110000/433787 utterances processed
120000/433787 utterances processed
130000/433787 utterances processed
140000/433787 utterances processed
150000/433787 utterances processed
160000/433787 utterances processed
170000/433787 utterances processed
180000/433787 utterances processed
190000/433787 utterances processed
200000/433787 utterances processed
210000/433787 utterances processed
220000/433787 utterances processed
230000/433787 utterances processed
240000/433787 utterances processed
250000/433787 utterances processed
260000/433787 utterances processed
270000/433787 utterances processed
280000/433787 utterances processed
290000/433787 utterances proc

We can then train a new `PhrasingMotifs` model that finds phrasings with the nouns still included.

In [33]:
noun_pm_model = PhrasingMotifs('motifs_full','question_arcs_full',min_support=100,
                               fit_filter=question_filter, 
                          verbosity=VERBOSITY)
noun_pm_model.fit(corpus)

counting frequent itemsets for 325339 sets
	first pass: counting itemsets up to and including 5 items large
	first pass: 10000/325339 sets processed
	first pass: 20000/325339 sets processed
	first pass: 30000/325339 sets processed
	first pass: 40000/325339 sets processed
	first pass: 50000/325339 sets processed
	first pass: 60000/325339 sets processed
	first pass: 70000/325339 sets processed
	first pass: 80000/325339 sets processed
	first pass: 90000/325339 sets processed
	first pass: 100000/325339 sets processed
	first pass: 110000/325339 sets processed
	first pass: 120000/325339 sets processed
	first pass: 130000/325339 sets processed
	first pass: 140000/325339 sets processed
	first pass: 150000/325339 sets processed
	first pass: 160000/325339 sets processed
	first pass: 170000/325339 sets processed
	first pass: 180000/325339 sets processed
	first pass: 190000/325339 sets processed
	first pass: 200000/325339 sets processed
	first pass: 210000/325339 sets processed
	first pass: 220000

The most common phrasings, of course, won't be very topic-specific (unless people talk about yachts very very frequently in parliament). However, we do see that phrasings now reflect the pronoun used (which may be troublesome if we believe that "Does _he_ agree" and "Does _she_ agree" are getting at similar things).

In [34]:
noun_pm_model.print_top_phrasings(25)

('*',) 325339
('will>*',) 70226
('does>*',) 61032
('is_*',) 57964
('is>*',) 45268
('is>*', 'is_*') 42850
('agree_*',) 36108
('agree_does',) 33704
('agree_*', 'agree_does') 33704
('agree_*', 'does>*') 30012
('agree_does', 'does>*') 29987
('agree_*', 'agree_does', 'does>*') 29987
('will>the',) 26218
('will>*', 'will>the') 26218
('will>he',) 23049
('will>*', 'will>he') 23049
('is_aware',) 22063
('is_*', 'is_aware') 22063
('does>the',) 20932
('does>*', 'does>the') 20932
('what>*',) 20791
('is>*', 'is_aware') 20707
('is>*', 'is_*', 'is_aware') 20707
('does>he',) 16417
('does>*', 'does>he') 16417


Here are the sink phrasings for our example utterance from earlier, comparing against the noun-less run:

In [35]:
utt = noun_pm_model.transform_utterance(utt)

In [88]:
utt.get_info('motifs__sink')

['agree_*__does>*', 'agree_*__agree_also', 'as>* share_*__share_does']

In [36]:
utt.get_info('motifs_full__sink')

['agree_*__agree_hon',
 'agree_*__agree_also__agree_he',
 'as>* share_*__share_hon']

We see that we get this extra "hon" -- which actually stands for "honourable [member]" -- an artefact of parliamentary etiquette. 

For our particular dataset, removing nouns has the benefit of removing most of these etiquette-related words. However, you may also imagine cases where nouns actually carry a lot of useful information about rhetorical intent (including in this domain -- one could argue that asking about a person versus asking about a department is a strong signal of trying to get at different things, for instance). As such, noun-removal is something that you may want to play around with, and/or try to improve upon. 

## PromptTypes

As we intuited above, "do you agree" and "do you share my opinion" are both getting at similar intentions. However, extracting these phrasings alone won't allow us to make this association. Rather, our strategy will be to produce vector representations of them which encode this similarity. Clustering these representations then gives us different "types of question".

Our key intuition here is that questions with similar intentions will tend to be answered in similiar ways. Thus, "do you agree" and "do you share" may both often be answered with "yes, I agree"; if tomorrow I asked a new question of this ilk ("do you agree that we should invest in planes, instead of yachts"), I might be expecting a similar sort of answer. 

For a full explanation of this idea, and how we operationalized it, you can read our paper. In ConvoKit, we implement this methodology of producing vector representations and clusterings via the `PromptTypes` transformer:

In [90]:
from convokit.prompt_types import PromptTypes

`PromptTypes` will train a model -- a low-dimensional embedding, along with a k-means clustering -- by using question-answer pairs as input. 

In [37]:
def question_filter(utt, aux_input={}):
    return utt.meta['is_question']
def response_filter(utt, aux_input={}):
    return (not utt.meta['is_question']) and (utt.reply_to is not None)

We initialize `pt` with the following arguments:

* `n_types=8`: we want to infer 8 types of questions.
* `prompt_field='motifs'`: we want to encode questions in terms of the phrasing motifs we extracted above. thus, `pt` will produce representations of these motifs (rather than, e.g., the raw tokens in a question)
* `ref_field='arcs_censored'`: we will encode responses in terms of the noun-less arcs we extracted above (in practice, this appears to work better than using phrasings of responses as well, perhaps because responses are noisier)
* `prompt_filter=question_filter` and `ref_filter=response_filter`: To tell the transformer what counts as a question and an answer, we will pass the constructor the above filters (i.e., boolean functions). Note that in a less questions-heavy dataset, we could omit these filters and hence infer types of "prompts" beyond questions.
* `prompt_transform_field='motifs__sink'`: while we want to come up with a representation of _all_ phrasing motifs, when we produce a vector representation of a _particular_ utterance we want to use the most finely-specified phrasing.

There are some other arguments you can set, which are listed in the docstring. 



In [39]:
pt = PromptTypes(n_types=8, prompt_field='motifs', ref_field='arcs_censored', 
                 prompt_filter=question_filter, ref_filter=response_filter,
                 prompt_transform_field='motifs__sink',
                 output_field='prompt_types',
    random_state=1000, verbosity=1)

We can fit `pt` to the corpus -- that is, learn the associations between question phrasings and response dependency arcs that allow us to produce our vector representations, as well as a clsutering of these representations that gives us our different question types.

In [40]:
pt.fit(corpus)

fitting 195441 input pairs
fitting ref tfidf model
fitting prompt tfidf model
fitting svd model
fitting 8 prompt types


calling `display_type()` as below will print the question phrasings, response arcs, and prototypical questions and responses that are associated with each inferred type of question. We will examine some of these types more closely by way of examples, below.

In [41]:
for i in range(8):
    print(i)
    pt.display_type(i, corpus=corpus, k=15)
    print('\n\n')

0
top prompt:


,0,1,2,3,4,5,6,7,type_id
made_*,0.617322,1.114613,1.255320,1.077988,1.115492,1.263859,1.081724,1.076337,0.0
made_*__made_in,0.660890,1.095649,1.298649,1.096428,1.102150,1.168903,1.113446,1.046065,0.0
made_*__made_to,0.671795,1.226338,1.221120,1.103266,1.175426,1.386013,1.211334,1.137368,0.0
made_*__made_been,0.673816,1.122863,1.279368,1.140499,1.174720,1.262665,1.173846,1.110745,0.0
in>*__tell_*,0.675825,1.177443,0.993464,0.845159,1.261066,1.330201,0.980830,1.112974,0.0
made_*__made_what,0.677538,1.119886,1.356694,1.105855,0.952826,1.247374,1.204205,1.134827,0.0
made_*__what>*,0.689943,1.144744,1.361479,1.149020,0.989999,1.245750,1.222378,1.134689,0.0
made_*__made_been__made_what,0.692419,1.102368,1.341362,1.149741,1.046976,1.222434,1.223794,1.127061,0.0
happen_*__happen_will,0.696431,1.201901,1.109235,0.865723,1.115287,1.236088,1.052395,1.155658,0.0
made_*__made_has,0.698737,1.121069,1.307046,1.132155,1.104679,1.305834,1.219281,1.176971,0.0


top response:


,0,1,2,3,4,5,6,7,type_id
am_at,0.735826,0.929084,1.207077,1.021880,1.233163,1.220739,1.067730,1.056635,0.0
known_*,0.740341,1.225947,1.118957,1.039590,1.239562,1.274925,1.072987,1.153537,0.0
place_*,0.781089,1.046100,1.162646,1.036335,1.122966,1.157035,1.093920,1.046621,0.0
can_*,0.785580,1.102952,1.203068,0.991884,0.976701,1.084446,1.067260,1.061940,0.0
assure_have,0.791607,0.941692,1.299600,1.075591,0.992283,0.971913,1.023242,0.914936,0.0
was_made,0.793669,1.190258,0.974625,0.907933,1.198473,1.178469,0.935950,1.151902,0.0
give_can,0.796445,1.035412,1.152593,0.875637,1.130717,1.228405,1.071766,1.069094,0.0
make_shall,0.798254,0.875871,1.177812,0.902875,1.126980,0.959178,0.824072,0.900683,0.0
write_shall,0.806027,1.158947,1.141040,1.088976,1.183973,1.271158,1.202233,1.168124,0.0
write_with,0.806967,1.158069,1.139075,1.083898,1.179597,1.273140,1.203698,1.170615,0.0


top prompts:
1996-02-06a.122.5 As the House will be interested today in the change of personnel at British Gas , what direct contacts have been made with British Gas on the laying of the pipeline ? What issues were discussed ? On what dates were meetings held and what recommendations , if any , were made by the Ministry of Defence ?
['as>* made_*__made_be made_*__made_been__made_have made_*__made_been__made_what made_*__made_have__made_what made_*__made_on__made_what', 'discussed_* what>*', 'made_*__made_on on>*']

2010-01-19c.145.1 In the context of Iran 's nuclear power , be it civil or for weapons , have we made any progress in persuading Moscow that an Iranian nuclear weapon would be much more immediately threatening to the Russians than it would be to us in western Europe ?
['in>* made_*__made_be made_*__made_have made_*__made_in']

2011-05-05c.764.8 The Secretary of State made a point about less prescriptive service requirements , but will he give a guarantee that stations such a

...

top prompts:
1987-11-02a.647.10 When that time comes , will the Minister and the Lord President bear in mind that schedule 4(5 ) to the Medical Act 1983 gives them powers to require further review ? Is the Minister aware that , in a recent note to Members of this House , the chairman of the General Medical Council said that the council 's principal task always has been , that of informing and protecting the public " . Would it not be a good thing to review the rules in the light of those excellent precepts , especially paragraph 60 of the current statutory instrument relating to the availablility of transcripts of the hearings of the GMC ? Is the Minister aware that , in a recent tragic case , the wife of a deceased gentleman was asked to pay £ 400 for the transcript of the case heard by that council ? Does he think that that charge needs review , and is it in line with the precepts that I have just quoted ?
['when>*', 'be_*__be_not__be_would__would>*', 'is_*__is_aware', 'does>*__is_* 

,0,1,2,3,4,5,6,7,type_id
learned_*__will_*,1.040354,0.909116,1.152059,1.154805,1.299710,0.841406,0.800129,0.543683,7.0
learned_*__will>*,1.035450,0.893771,1.162074,1.151075,1.296721,0.851857,0.821541,0.543967,7.0
bear_*__bear_in__in>*,1.083888,0.956930,1.217765,1.151470,1.223059,0.979592,0.977491,0.550360,7.0
draw_*__will>*,1.023994,0.910458,1.137336,1.057207,1.176448,0.939863,0.871607,0.551262,7.0
draw_*__draw_will,1.041282,0.913071,1.145222,1.072801,1.181680,0.902788,0.874199,0.559115,7.0
convey_*__convey_to,1.087247,0.962480,1.170417,1.086914,1.231473,1.015065,0.989478,0.565904,7.0
will_*,1.002983,0.847088,1.124928,1.101191,1.312992,0.835394,0.721011,0.572541,7.0
convey_*__convey_to__convey_will,1.112289,1.025614,1.149763,1.127112,1.216016,0.986942,0.993735,0.591980,7.0
will>*__will_*,1.002939,0.826671,1.157892,1.125611,1.321378,0.866592,0.762808,0.598355,7.0
does_*__learned_*__learned_accept,1.103301,1.001168,1.067266,1.123401,1.323748,0.872465,0.735771,0.610948,7.0


top response:


,0,1,2,3,4,5,6,7,type_id
note_says,1.002260,0.963592,1.113321,1.035899,1.287303,0.980794,0.803377,0.604191,7.0
emphasise_*,1.039990,0.851872,1.190487,1.116548,1.260419,0.919627,0.807381,0.610986,7.0
note_*,1.048791,0.923442,1.065397,1.080076,1.350072,0.978475,0.758877,0.618893,7.0
learned_*,0.995554,0.941518,1.052428,0.994120,1.246673,0.778027,0.668100,0.621992,7.0
is_consider,0.981082,0.827353,1.229512,1.014184,1.142960,0.964230,0.923627,0.635198,7.0
be_important,1.087818,0.868260,1.168884,1.051799,1.141418,0.857980,0.824221,0.635525,7.0
are_always,1.076061,0.922007,1.159046,1.144859,1.267237,1.026292,0.901265,0.641044,7.0
convey_*,1.120694,0.941051,1.196546,1.141500,1.273676,1.077050,1.017874,0.641890,7.0
consider_is,0.964139,0.815137,1.196858,1.010101,1.241343,1.014127,0.817837,0.647248,7.0
consider_must,1.036941,0.897222,1.186344,1.111420,1.281730,0.963902,0.848750,0.649230,7.0


top prompts:
1980-05-15a.1743.0 Will my right hon Friend take time to study the difference in pay settlements between the private sector and the public services and public monopolies , especially the water authorities ? Will she bear in mind that our constituents , especially mine , are increasingly unable to pay for the enormous cost of water and sewage treatment ? Will she draw the appropriate conclusions ?
['take_*__will>*', 'bear_*__will>*', 'draw_*__will>*']

1997-07-03a.409.6 I thank the hon Lady for that answer . Will she acknowledge that it is a matter of great importance that some 23 per cent . of the population now live in rural areas ? In particular , will she bear in mind the importance of not getting business links too hung up on the size of firms , which typically is smaller in rural areas ? Will she make use , wherever possible , of other institutions such as the Ministry of Agriculture , Fisheries and Food and the Rural Development Commission , and make the maximum poss

When this trained model is used to transform a corpus, it will output several representations or features associated with each utterance.

In [42]:
utt = pt.transform_utterance(utt)

A vector representation encapsulating the utterance's rhetorical intent (in short, an embedding of the utterance based on the responses associated with questions containing its constituent phrasings):

In [43]:
utt.get_info('prompt_types__prompt_repr')

[-0.1697405599956893,
 0.03632750344898307,
 -0.16960577009269515,
 0.13467740850273394,
 -0.3033313505509189,
 -0.017352642005944222,
 -0.21475294465096412,
 -0.13411435880513273,
 0.17912876584377993,
 0.021433472222178843,
 -0.3530617501679388,
 -0.24195307167099495,
 -0.06836710808789839,
 -0.18317690396891484,
 -0.03399566433694401,
 -0.030218990740537546,
 -0.41354193763410346,
 -0.06592637574242852,
 -0.10894426409285005,
 -0.02459589626813662,
 -0.04600034458088039,
 -0.5460362677858065,
 0.13112554982604735,
 0.07216726420879413]

The distance between the vector of that utterance and the centroid of each cluster, i.e., type of question:

In [44]:
utt.get_info('prompt_types__prompt_dists.8')

[1.1457395346487957,
 0.9504055397170942,
 1.1230208124602417,
 1.1222589550014062,
 1.1162666884161712,
 0.38896782295615245,
 0.7627830151500072,
 0.8520923241206688]

The particular type of question this utterance is, as well as how close it is to the centroid of that particular cluster (roughly, how well it fits that question type):

In [45]:
utt.get_info('prompt_types__prompt_type.8')

5.0

In [46]:
utt.get_info('prompt_types__prompt_type_dist.8')

0.38896782295615245

Here, we see that our running example is of question type 5, which is exemplified by phrasings like `does [the Minister] agree...` -- we may characterize the entire cluster as encapsulating "agreeing" questions which are perhaps asked helpfully to bolster the answerer's reputation.

In [94]:
pt.display_type(5, k=15)

top prompt:


,0,1,2,3,4,5,6,7,type_id
agree_*__agree_is,1.190007,1.063678,1.130037,1.229631,1.120525,0.389741,0.868799,0.966774,5.0
agree_*__agree_be__does>*,1.146346,1.020134,1.141136,1.201449,1.132941,0.396055,0.800297,0.897559,5.0
agree_*__agree_be,1.141823,1.022926,1.140163,1.191639,1.138213,0.396684,0.790162,0.890782,5.0
agree_*__agree_is__does>*,1.187284,1.068057,1.132884,1.235290,1.114640,0.397564,0.876472,0.968414,5.0
agree_*__agree_have,1.160925,1.060035,1.154290,1.228834,1.128729,0.437026,0.867084,0.967709,5.0
agree_*__agree_are,1.199042,1.094654,1.126822,1.249057,1.161698,0.444698,0.868741,0.951413,5.0
agree_*__agree_does__agree_have__does>*,1.146106,1.091873,1.136289,1.216599,1.115092,0.450785,0.852474,0.969716,5.0
agree_*__agree_are__agree_does__does>*,1.200686,1.101353,1.124174,1.253320,1.158171,0.456768,0.878216,0.955740,5.0
agree_*__agree_also,1.185051,1.136070,1.117248,1.261808,1.174689,0.466548,0.904168,1.037751,5.0
continue_*__will>*,1.156545,1.037004,1.200064,1.197731,0.987681,0.472096,0.966770,0.982168,5.0


top response:


,0,1,2,3,4,5,6,7,type_id
agree_certainly,1.179935,1.072132,1.193617,1.288381,1.101887,0.461667,0.959144,0.983017,5.0
agree_however,1.170462,1.077594,1.185802,1.281559,1.116320,0.468359,0.939135,1.006986,5.0
agree_is,1.173939,1.072148,1.197891,1.286641,1.095410,0.468887,0.956498,0.996706,5.0
agree_will,1.181074,1.043084,1.202556,1.287871,1.122069,0.476162,0.951444,1.012685,5.0
agree_absolutely,1.194568,1.063192,1.211595,1.289561,1.068197,0.477591,0.993222,0.993978,5.0
agree_also,1.194003,1.079655,1.187382,1.290039,1.097132,0.478019,0.954352,1.003185,5.0
agree_wholeheartedly,1.185445,1.092127,1.183965,1.275667,1.089317,0.479565,0.971246,1.030695,5.0
is_also,1.198891,1.047256,1.096212,1.099916,1.003657,0.480978,0.882228,0.999332,5.0
agree_be,1.163662,1.079722,1.196200,1.283851,1.103804,0.481309,0.949732,1.004706,5.0
is_reduce,1.113240,1.054234,1.080224,1.156094,1.134554,0.482403,0.752575,0.906337,5.0


We can transform the other utterances in the corpus as such:

In [47]:
corpus = pt.transform(corpus)

This utterance is of type 4: perhaps more information-seeking and querying for an update ("what steps is the Government taking, what are they doing to ensure", etc)

In [100]:
utt1.text

'Given what the Foreign Secretary has said about the importance of the Iran discussions on the nuclear agreement , what is he doing to ensure greater clarity about the baselines , the extent of the inspection regime and the consequences of infringement ? Given that the agreement will allow advanced centrifuge , the infringements might arrive a little earlier than anticipated .'

In [101]:
utt1.get_info('motifs__sink')

['doing_*__doing_ensure__doing_is doing_*__doing_is__doing_what given>*']

In [102]:
utt1.get_info('prompt_types__prompt_type.8')

4.0

In [103]:
pt.display_type(4, k=15)

top prompt:


,0,1,2,3,4,5,6,7,type_id
doing_*__what>*,1.190520,1.170917,1.306277,1.159109,0.489782,1.163157,1.381528,1.286072,4.0
doing_*,1.197745,1.170508,1.281262,1.150488,0.505397,1.147638,1.346396,1.272227,4.0
taking_*__taking_is__what>*,1.130973,1.184592,1.342540,1.172115,0.508758,1.163605,1.407868,1.246767,4.0
doing_*__doing_is__what>*,1.197377,1.191344,1.312856,1.182806,0.529712,1.214734,1.420265,1.312049,4.0
take_*__take_what,1.161917,1.001976,1.343445,1.153361,0.534803,1.092679,1.296199,1.190162,4.0
taking_*__taking_are,1.091065,1.213732,1.364810,1.182392,0.535213,1.194818,1.402069,1.239454,4.0
taking_*,1.137747,1.225600,1.351025,1.198333,0.536375,1.192687,1.420150,1.258012,4.0
will>*__work_*__work_with,1.058973,0.960279,1.382445,1.148531,0.537547,1.015651,1.253309,1.120588,4.0
taking_*__what>*,1.088500,1.218316,1.368173,1.186242,0.540326,1.199291,1.414438,1.255039,4.0
doing_*__doing_is,1.206442,1.211483,1.288720,1.173522,0.542670,1.210846,1.397774,1.301012,4.0


top response:


,0,1,2,3,4,5,6,7,type_id
through>*,1.173402,1.236664,1.337521,1.241417,0.640292,1.061877,1.373256,1.284117,4.0
is_working,1.119196,1.145401,1.280114,1.148700,0.645109,0.976848,1.213952,1.210242,4.0
ensuring_is,1.199215,1.082640,1.248994,1.137753,0.649250,0.959081,1.229484,1.118896,4.0
supporting_are,1.217661,1.240021,1.242036,1.170911,0.652102,1.184466,1.366115,1.295775,4.0
working_on,1.135818,1.218759,1.303140,1.203003,0.663693,1.310517,1.381095,1.280683,4.0
supporting_*,1.222778,1.252930,1.254030,1.190700,0.666032,1.201988,1.390020,1.322857,4.0
ensuring_*,1.178767,1.116399,1.228789,1.094961,0.667111,0.930856,1.186430,1.159537,4.0
working_are,1.137959,1.223427,1.296334,1.199166,0.669878,1.314324,1.379198,1.282322,4.0
working_with,1.137950,1.221931,1.297881,1.199787,0.672097,1.316745,1.380642,1.281343,4.0
working_*,1.139951,1.223877,1.296162,1.200937,0.672468,1.314581,1.380422,1.282824,4.0


This utterance, on the other hand is a lot more aggressive -- perhaps _accusatory_ to the ends of putting the answerer on the spot ("will the secretary admit that the policy is a failure?")

In [106]:
utt2 = corpus.get_utterance('1987-03-04a.857.5')

In [107]:
utt2.text

'Will the Secretary of State stop giving us what is called in the pop record industry a remix of alibis , excuses and gimmicks ? Will he admit that the number of homes built to rent last year by local authorities was the lowest in 62 years , that the housing investment programme net of capital receipts was the lowest in real terms since HIPs were invented and that , even during the past three years the number of repair and improvement grants , which would bring some private homes back into use , have dropped by 100,000 ? Does not the right hon Gentleman understand that , if the private owner and the local authority are starved of resources , we are left with lengthy queues , homelessness and all the other scandals of poor housing that exist today ?'

In [108]:
utt2.get_info('motifs__sink')

['stop_*__stop_will__will>*',
 'admit_*__admit_will__will>*',
 'does>*__does>not does>*__understand_*']

In [109]:
utt2.get_info('prompt_types__prompt_type.8')

2.0

In [111]:
pt.display_type(2, k=15)

top prompt:


,0,1,2,3,4,5,6,7,type_id
admit_*,1.206418,1.296074,0.572506,0.939985,1.397106,1.270348,0.955147,1.219785,2.0
why>*,1.115702,1.312999,0.582089,0.858220,1.386590,1.284560,0.915984,1.230090,2.0
admit_*__will>*,1.237791,1.316191,0.582742,0.985069,1.359790,1.245596,1.028147,1.264033,2.0
is>*__is_*__is_true,1.176470,1.325565,0.587340,1.017903,1.480818,1.142600,0.866422,1.106685,2.0
explain_*,1.085886,1.258692,0.588059,0.838155,1.366495,1.271532,0.923838,1.209288,2.0
explain_*__explain_will,1.088246,1.301927,0.591164,0.921589,1.375481,1.189579,0.886641,1.188762,2.0
is_*__why>*,1.193965,1.284715,0.606379,0.918270,1.366362,1.175479,0.864822,1.226070,2.0
is_*__is_true,1.176367,1.346108,0.608337,1.032375,1.492904,1.184561,0.916077,1.161737,2.0
does>*__realise_*__realise_does__realise_not,1.164062,1.347022,0.614524,0.958736,1.390545,1.253235,0.934875,1.149033,2.0
admit_*__admit_will__will>*,1.245443,1.312688,0.616380,0.972787,1.349487,1.277325,1.075932,1.295452,2.0


top response:


,0,1,2,3,4,5,6,7,type_id
wonder_*,1.185317,1.280951,0.598627,0.862517,1.313988,1.111706,0.882447,1.191012,2.0
failed_*,1.212231,1.332105,0.623728,0.967290,1.340943,1.128558,0.946383,1.264899,2.0
were_*,1.215991,1.371217,0.639946,1.062895,1.352447,1.074506,0.939683,1.208665,2.0
is_wrong,1.176518,1.387354,0.657307,0.968697,1.312063,1.154070,0.965930,1.260657,2.0
instead>*,1.184016,1.264147,0.684000,0.842224,1.264422,1.243630,1.013340,1.220351,2.0
was_*,1.183702,1.181505,0.690253,0.927966,1.289817,0.881450,0.743510,1.117129,2.0
were_there,1.218082,1.395072,0.693541,1.081200,1.356462,1.155589,1.012219,1.255169,2.0
remind_*,1.158828,1.090797,0.697386,0.903524,1.360224,1.070433,0.727951,0.965714,2.0
talks_*,1.246355,1.231195,0.698599,0.897893,1.340013,1.264851,1.045402,1.242012,2.0
talks_about,1.239044,1.226465,0.711066,0.884990,1.340720,1.301715,1.057403,1.244414,2.0


Inspecting the other types should hopefully give you an intuition for the range of questions that tend to be asked in parliament, as well as the coherence of these types (which align fairly well with the output of the paper, even under different random seeds and small implementation tweaks). 

### a few caveats and potential modifications

One thorn in our sides might be that the model occasionally gets caught up on very generic motifs e.g., `'is>*'`, and as such, will fit many questions to the type containing `'is>*'` instead of going with a better signal; various optional parameters detailed in the documentation may provide incomplete solutions to this. Another caveat is that while this model allows us to associate together lexically-diverging phrasings (e.g., "will the Minister admit" and "does the Minister not realise" both serve to be accusatory towards the Minister), we are ultimately relying on the fact that our domain has a sufficient amount of lexical regularity (e.g., the institutional norms of how people talk in parliament) -- we might need to be cleverer when dealing with noisier settings where this regularity isn't guaranteed (like social media data). 

Finally, as a data-specific note, cluster 7 may be a result of the parser assuming that "Will the learned Gentleman please answer my question?" has "learned" as the root verb -- an artefact of parliamentary discourse we haven't handled. You may wish to play around with this by modifying how the data is preprocessed.

### model persistence

We can save our trained `pt_model` to disk for later use:

In [51]:
import os

In [52]:
pt.dump_model(os.path.join(ROOT_DIR, 'pt_model'))

dumping embedding model
dumping training embeddings
dumping type model 8


In broad strokes, what's loaded to disk is:

* TfIdf models that store the distribution of phrasings and arcs in the training data;
* SVD models that allow us to map raw phrasing/arc counts to vector representations;
* a KMeans model to cluster vector representations.

In [112]:
pt_model_dir = os.path.join(ROOT_DIR, 'pt_model')
!ls $pt_model_dir

km_model.8.joblib	   svd_model.joblib	   train_ref_ids.npy
prompt_df.8.tsv		   train_prompt_df.8.tsv   train_ref_vects.npy
prompt_tfidf_model.joblib  train_prompt_ids.npy    U_prompt.npy
ref_df.8.tsv		   train_prompt_vects.npy  U_ref.npy
ref_tfidf_model.joblib	   train_ref_df.8.tsv


Initializing a new `PromptTypes` model and loading our saved model then allows us to use it again:

In [53]:
new_pt = PromptTypes(prompt_field='motifs', ref_field='arcs_censored', 
                 prompt_filter=question_filter, ref_filter=response_filter,
                 prompt_transform_field='motifs__sink',
                 output_field='prompt_types_new', prompt__tfidf_min_df=100,
                 ref__tfidf_min_df=100, 
    random_state=1000, verbosity=1)

In [55]:
new_pt.load_model(pt_model_dir)

loading embedding model
loading training embeddings
loading type model 8


In [56]:
utt = new_pt.transform_utterance(utt)

In [57]:
utt.get_info('prompt_types_new__prompt_type.8')

5.0

## examples of potential variations

### trying other numbers of prompt types:

Calling `refit_types(n)` will retrain the clustering component of the `PromptType` model to infer a different number of types. Suppose we only wanted 4 types of questions:

In [58]:
pt.refit_types(4)

fitting 4 prompt types


In [60]:
for i in range(4):
    print(i)
    pt.display_type(i, type_key=4, k=15)
    print('\n\n')

0
top prompt:


,0,1,2,3,type_id
taking_*__taking_is__what>*,0.609794,1.229558,1.186419,1.298549,0.0
will>*__work_*__work_with,0.616848,1.065764,1.037907,1.312523,0.0
doing_*__what>*,0.617565,1.227830,1.205728,1.263668,0.0
taking_*__taking_are,0.617644,1.241877,1.180574,1.316730,0.0
taking_*__what>*,0.618966,1.253273,1.186392,1.320488,0.0
do_*__do_help,0.622422,1.196425,1.106535,1.259926,0.0
done_*__done_being,0.626201,1.145207,1.112165,1.262309,0.0
doing_*,0.631977,1.204957,1.199373,1.240347,0.0
taking_*,0.632458,1.249054,1.210879,1.310954,0.0
do_*__do_can__do_what,0.635238,1.191807,1.126391,1.271694,0.0


top response:


,0,1,2,3,type_id
is_working,0.696911,1.046191,1.149283,1.224687,0.0
ensure_to,0.699242,0.943430,1.050312,1.220637,0.0
raises_*,0.700467,1.075396,0.894595,1.291588,0.0
through>*,0.726525,1.164724,1.233682,1.307317,0.0
taking_in,0.730056,1.021793,1.103756,1.190225,0.0
ensuring_*,0.733671,1.003093,1.123217,1.184374,0.0
ensuring_is,0.736937,1.023703,1.124906,1.211085,0.0
met_discuss,0.737335,1.131090,0.964341,1.336538,0.0
supporting_are,0.738320,1.238487,1.234494,1.219720,0.0
working_on,0.741080,1.314051,1.196169,1.268169,0.0





1
top prompt:


...

,0,1,2,3,type_id
understand_*,1.103667,0.885863,0.660863,0.970153,2.0
understand_will,1.086387,1.002950,0.687529,0.995473,2.0
appreciate_will,1.125908,1.005765,0.691698,1.046429,2.0
appreciate_*,1.155661,0.920503,0.695537,1.034463,2.0
consider_be,1.134915,0.943910,0.700464,1.133897,2.0
be_shall,1.053164,0.756672,0.710572,1.011730,2.0
am_aware,1.068362,1.024942,0.712944,1.168109,2.0
however>*,1.144221,0.771370,0.714712,0.958140,2.0
be_happy,1.021427,0.767231,0.720347,1.062687,2.0
consider_is,1.159007,0.861353,0.723786,1.112034,2.0





3
top prompt:


,0,1,2,3,type_id
why>*,1.308495,1.181707,1.085796,0.603725,3.0
explain_*,1.276329,1.167333,1.059989,0.603931,3.0
why>*__why>does,1.303258,1.035683,1.014513,0.617478,3.0
explain_*__explain_will,1.295929,1.098273,1.084987,0.632064,3.0
is_*__why>*,1.303829,1.090458,1.113158,0.640935,3.0
admit_*,1.331656,1.173335,1.123428,0.644744,3.0
has>*__has>not,1.310626,0.937279,0.988593,0.648477,3.0
explain_*__explain_is,1.229206,1.104504,1.027583,0.649819,3.0
explain_*__will>*,1.260808,1.065725,1.082579,0.651681,3.0
is>*__is_*__is_true,1.404483,1.040420,1.115109,0.664331,3.0


top response:


,0,1,2,3,type_id
wonder_*,1.249856,1.044783,1.103523,0.628782,3.0
is>*,1.168523,1.007722,0.988924,0.675574,3.0
failed_*,1.284047,1.085757,1.170842,0.678967,3.0
suggest_*,1.232084,1.019900,0.965210,0.682459,3.0
says_*,1.171131,1.067435,1.002317,0.694058,3.0
instead>*,1.207184,1.179012,1.098313,0.694607,3.0
was_*,1.238018,0.840570,1.058230,0.699676,3.0
remind_*,1.298467,0.938198,0.949616,0.699823,3.0
is_wrong,1.269154,1.105955,1.172906,0.706003,3.0
is_what,1.161270,1.037770,1.075061,0.707161,3.0


### trying other input formats

We may also experiment with different representations of the input text -- for instance, in lieu of using phrasing motifs we may instead pass questions into the model as just the raw arcs, similar to the responses. This can be modified by changing the `prompt_field` argument:

In [113]:
pt_arcs = PromptTypes(prompt_field='arcs_censored', ref_field='arcs_censored', 
                 prompt_filter=question_filter, ref_filter=response_filter,
                 prompt_transform_field='arcs_censored',
                 output_field='prompt_types_arcs', prompt__tfidf_min_df=100,
                 ref__tfidf_min_df=100, n_types=8,
    random_state=1000, verbosity=1)

In [114]:
pt_arcs.fit(corpus)

fitting 214798 input pairs
fitting ref tfidf model
fitting prompt tfidf model
fitting svd model
fitting 8 prompt types


In [116]:
for i in range(8):
    print(i)
    pt_arcs.display_type(i,  k=10)
    print('\n\n')

0
top prompt:


,0,1,2,3,4,5,6,7,type_id
can>*,0.609116,1.044252,1.033734,0.851297,1.165797,1.009898,1.007405,1.054094,0.0
say_can,0.652671,1.125536,1.068430,0.889078,1.266746,1.047749,1.103901,1.090667,0.0
tell_*,0.661456,1.129387,0.953136,0.856639,1.293325,1.119897,1.155482,0.992021,0.0
made_for,0.662053,1.164619,0.952213,0.955347,1.051719,0.891899,0.997746,1.130847,0.0
give_*,0.665993,1.108280,1.014296,0.813365,1.143359,0.801453,0.939930,1.197438,0.0
give_can,0.666352,0.945825,1.083009,0.789525,1.099114,0.961858,0.974658,1.136283,0.0
tell_will,0.668334,1.119809,0.936050,0.846824,1.315597,1.114596,1.164083,1.024658,0.0
give_on,0.672531,0.985887,1.101895,0.874927,1.172106,1.058131,1.007081,1.170271,0.0
expect_do,0.694729,1.171479,0.982923,0.835003,1.100034,0.931635,0.991873,1.151189,0.0
tell_can,0.697686,1.153934,1.061992,0.974253,1.311530,1.195676,1.160011,0.993638,0.0


top response:


,0,1,2,3,4,5,6,7,type_id
am_able,0.646785,1.098170,0.962517,0.872964,1.153355,0.911970,0.984667,1.139347,0.0
give_not,0.677452,1.063195,1.115032,0.933426,1.209333,1.064506,0.995220,1.076234,0.0
answer_*,0.678642,1.115913,1.017341,0.877308,1.289749,1.115105,1.096958,1.056636,0.0
answer_not,0.680949,1.111065,1.086076,0.949141,1.305902,1.125563,1.075835,1.092057,0.0
answer_can,0.686658,1.112905,1.087239,0.970444,1.300033,1.129931,1.086768,1.088089,0.0
have_not,0.689800,1.082363,1.037531,0.967961,1.079582,1.021271,1.021324,1.073268,0.0
give_can,0.692646,1.028448,1.154721,0.934014,1.168879,1.044568,1.004084,1.094822,0.0
undertake_*,0.695635,1.055936,1.150660,0.989665,1.134439,0.965506,0.873642,1.181397,0.0
tell_not,0.695791,1.026490,1.038474,0.907341,1.214771,1.096822,1.067981,1.028642,0.0
answer_will,0.697729,1.130367,0.935838,0.733411,1.227748,1.052175,1.086865,1.019216,0.0





1
top prompt:


...

,0,1,2,3,4,5,6,7,type_id
want_obviously,1.132746,1.025288,1.260912,1.036430,0.952353,0.888381,0.634628,1.176505,6.0
am_happy,1.031490,0.995484,1.270816,1.016258,0.874008,1.002797,0.651875,1.162853,6.0
am_always,1.092560,1.107887,1.166570,0.944404,0.959476,0.838730,0.653278,1.224491,6.0
raises_*,1.037261,0.719773,1.373454,1.114933,1.036623,1.056506,0.658898,1.260270,6.0
want_make,1.084712,0.923319,1.250548,0.956550,0.847335,1.030269,0.659323,1.154624,6.0
thank_*,1.135136,0.843192,1.357992,1.058280,0.826163,0.974123,0.672124,1.205623,6.0
want_give,1.052426,0.880642,1.230486,0.947574,0.819550,1.001802,0.676712,1.126525,6.0
thank_for,1.140063,0.836447,1.368714,1.068230,0.836829,0.988459,0.685237,1.208460,6.0
know_been,0.932715,1.080702,1.185565,1.045303,0.880846,0.888744,0.687020,1.179048,6.0
want_will,1.046792,0.947489,1.177959,0.889744,0.810550,0.944369,0.687210,1.101311,6.0





7
top prompt:


,0,1,2,3,4,5,6,7,type_id
since_*,1.204977,1.233116,0.978534,1.140813,1.161860,1.230927,1.257677,0.569861,7.0
fell_*,1.188856,1.208908,0.954174,1.039535,1.276057,1.299603,1.277657,0.586401,7.0
show_*,1.232587,1.264596,0.872948,1.120279,1.128625,1.235205,1.281634,0.593075,7.0
higher>*,1.201880,1.227956,0.965698,1.113849,1.133361,1.217782,1.237407,0.593701,7.0
risen_*,1.191418,1.247141,0.993018,1.154678,1.219707,1.283740,1.279646,0.601795,7.0
higher_*,1.186986,1.263328,0.924919,1.125214,1.082434,1.164154,1.247052,0.607989,7.0
fallen_*,1.224664,1.178423,1.101110,1.208755,1.238365,1.348327,1.298999,0.608869,7.0
fallen_has,1.216809,1.161739,1.135709,1.221443,1.222004,1.351368,1.298726,0.618473,7.0
fallen_by,1.189972,1.120609,1.123128,1.163305,1.169458,1.281215,1.230097,0.630365,7.0
rising_*,1.231992,1.177875,1.053844,1.147566,1.191554,1.269497,1.250988,0.632327,7.0


top response:


,0,1,2,3,4,5,6,7,type_id
rising_*,1.239914,1.199545,1.035662,1.187159,1.154295,1.236610,1.299108,0.586899,7.0
is_higher,1.257848,1.259956,0.981297,1.204614,1.111601,1.219320,1.311394,0.593472,7.0
is_down,1.200102,1.227429,1.015709,1.133334,1.232864,1.284627,1.296099,0.597229,7.0
rose_by,1.220997,1.227791,1.035028,1.228286,1.208716,1.282937,1.292786,0.600214,7.0
are_now,1.213837,1.159416,0.997061,1.136150,1.164207,1.301022,1.313702,0.604068,7.0
rising_is,1.227334,1.213439,1.018902,1.171369,1.165289,1.224533,1.285856,0.604345,7.0
rose_*,1.219812,1.235257,1.050710,1.229849,1.222976,1.289088,1.292472,0.605858,7.0
is_by,1.214416,1.204584,0.983280,1.124798,1.080587,1.151649,1.248702,0.607077,7.0
was_in,1.163913,1.254661,0.817196,1.052183,1.040283,1.155024,1.282360,0.609651,7.0
cut_have,1.241804,1.229279,0.968621,1.158152,1.240200,1.305202,1.320199,0.612180,7.0


## storing vector representations

As mentioned above, `PromptTypes` produces a few vector representations of utterances. For efficiency, rather than storing these representations attached to the utterance (as values in utterance.meta), we store them in a corpus-wide matrix.

`get_vect_repr(utterance_id, matrix name)` allows us to access the representation of a particular utterance:

In [117]:
corpus.get_vect_repr(test_utt_id, 'prompt_types__prompt_repr')

array([-0.16974056,  0.0363275 , -0.16960577,  0.13467741, -0.30333135,
       -0.01735264, -0.21475294, -0.13411436,  0.17912877,  0.02143347,
       -0.35306175, -0.24195307, -0.06836711, -0.1831769 , -0.03399566,
       -0.03021899, -0.41354194, -0.06592638, -0.10894426, -0.0245959 ,
       -0.04600034, -0.54603627,  0.13112555,  0.07216726])

To save all of these representations to disk, we can call the following:

In [73]:
corpus.dump_vector_reprs('prompt_types__prompt_repr')

This stores the representations (`vect_info.<FIELD NAME>.npy`) as a matrix, and the utterance IDs corresponding to each of the rows (`vect_info.<FIELD NAME>.keys`) -- both in the corpus directory.

In [74]:
ls $ROOT_DIR

conversations.json        pm_model/
corpus.json               pt_model/
index.json                users.json
info.arcs_censored.jsonl  utterances.jsonl
info.motifs.jsonl         vect_info.prompt_types__prompt_repr.keys
info.motifs__sink.jsonl   vect_info.prompt_types__prompt_repr.npy
info.parsed.jsonl


These vector representations can later be re-loaded:

In [75]:
new_corpus = convokit.Corpus(ROOT_DIR)

In [76]:
new_corpus.vector_reprs.keys()

dict_keys([])

In [77]:
new_corpus.load_vector_reprs('prompt_types__prompt_repr')

In [118]:
new_corpus.get_vect_repr(test_utt_id, 'prompt_types__prompt_repr')

array([-0.16974056,  0.0363275 , -0.16960577,  0.13467741, -0.30333135,
       -0.01735264, -0.21475294, -0.13411436,  0.17912877,  0.02143347,
       -0.35306175, -0.24195307, -0.06836711, -0.1831769 , -0.03399566,
       -0.03021899, -0.41354194, -0.06592638, -0.10894426, -0.0245959 ,
       -0.04600034, -0.54603627,  0.13112555,  0.07216726])